In [1]:
import torch
from nltk.tokenize import sent_tokenize
from datasets import load_dataset
from tqdm.notebook import tqdm
import pandas as pd

# The models the authors used:
from transformers import BertForMaskedLM, BertTokenizer

from blanc import BLANC_tune_summary, BLANC_tune_translation, add_results_to_json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Liora\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bert_checkpoint = "bert-base-uncased"
bert_model = BertForMaskedLM.from_pretrained(bert_checkpoint).to(DEVICE)
bert_tokenizer = BertTokenizer.from_pretrained(bert_checkpoint, do_lower_case=True)

mbert_checkpoint = "bert-base-multilingual-uncased"
mbert_model = BertForMaskedLM.from_pretrained(mbert_checkpoint).to(DEVICE)
mbert_tokenizer = BertTokenizer.from_pretrained(mbert_checkpoint, do_lower_case=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint

# BLANC tune for **summaries**

In [4]:
""" Datasets """

DailyNews_ds = load_dataset("json", data_files="../datasets/DailyNews_300.json", split="train")
DailyNews_ds

Dataset({
    features: ['text', 'annotators_ids', 'summary', 'scores'],
    num_rows: 300
})

In [5]:
""" Preprocessing """

summaries = DailyNews_ds["summary"]  # (List[str])
texts = DailyNews_ds[
    "text"
]  # (List[str]) each string is a paragraph made of a few sentences

# each text in texts is a list of sentences (each sentence is a string)
texts = [sent_tokenize(text.strip()) for text in texts]  # List[List[str]]
assert len(texts) == len(summaries) == 300

tokenized_texts = [
    [bert_tokenizer.tokenize(sentence) for sentence in text] for text in texts
]  # List[List[List[str]]]
tokenized_summaries = [
    bert_tokenizer.tokenize(summary) for summary in summaries
]  # [List[List[str]]]

In [7]:
BLANC_tune_summary(
        tokenized_texts[198], tokenized_summaries[198], bert_checkpoint, bert_model, bert_tokenizer, device=DEVICE)

hola
[]
coucou


0.0

In [ ]:
""" Running the Program """

tune_summary_scores = [
    BLANC_tune_summary(
        text, summary, bert_checkpoint, bert_model, bert_tokenizer, device=DEVICE
    )
    for text, summary in tqdm(
        zip(tokenized_texts, tokenized_summaries), total=len(tokenized_texts)
    )
]

# Saving the results
summary_data = {}
summary_data["BLANC_tune_summary"] = tune_summary_scores
add_results_to_json(summary_data)

# BLANC tune for **translations**

In [3]:
""" Datasets """

# English - French
en_fr_df = pd.read_csv("../datasets/en_to_fr_translations.csv", index_col=0)

# English - Persian (Farsi)
en_fa_ds = load_dataset("persiannlp/parsinlu_translation_en_fa", split="train")

In [4]:
""" Preprocessing (English - French)"""

# Tokenization
en_fr_sentences = [
    mbert_tokenizer.tokenize(sentence) for sentence in en_fr_df["en"]
]  # (List[List[str]])

en_fr_translations = [
    mbert_tokenizer.tokenize(translation) for translation in en_fr_df["fr"]
]  # (List[List[str]])


""" Preprocessing (English - Persian (Farsi)) """

# Removing the 'category' column
en_fa_ds = en_fa_ds.remove_columns(["category"])

# Removing list encapsulation
en_fa_ds = en_fa_ds.map(lambda example: {"targets": example["targets"][0]}, num_proc=4)

# Filtering out:
# - rows with the '\u200c' symbol,
# - those where the length of either source or targets is less than a threshold
# - Headlines (ending in 'Global Voices') --> because they are very short and the 'Global Voices' part is never translated
length_threshold = 30
filtered_en_fa_ds = en_fa_ds.filter(
    lambda example: "\u200c" not in example["targets"]
    and len(example["source"]) >= 30
    and len(example["targets"]) >= 30
    and "Global Voices" not in example["source"],
    num_proc=4,
)

en_fa_ds = (
    filtered_en_fa_ds.rename_column("source", "sentence")
    .rename_column("targets", "translation")
    .select(range(300))
)

# Tokenization
en_fa_sentences = [
    mbert_tokenizer.tokenize(sentence) for sentence in en_fa_ds["sentence"]
]  # (List[List[str]])

en_fa_translations = [
    mbert_tokenizer.tokenize(translation) for translation in en_fa_ds["translation"]
]  # (List[List[str]])

In [5]:
tune_en_fr_scores = [
    BLANC_tune_translation(
        sentences,
        translations,
        mbert_checkpoint,
        mbert_model,
        mbert_tokenizer,
        device=DEVICE,
    )
    for sentences, translations in tqdm(
        zip(en_fr_sentences, en_fr_translations), total=len(en_fr_sentences)
    )
]

# Saving the results
en_fr_data = {}
en_fr_data["BLANC_tune_en_fr_translation"] = tune_en_fr_scores
add_results_to_json(en_fr_data)

  0%|          | 0/100 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/15 [00:00<?, ?it/s]

{'train_runtime': 28.5355, 'train_samples_per_second': 4.205, 'train_steps_per_second': 0.526, 'train_loss': 0.2070212682088216, 'epoch': 3.0}


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/15 [00:00<?, ?it/s]

{'train_runtime': 17.5932, 'train_samples_per_second': 6.821, 'train_steps_per_second': 0.853, 'train_loss': 0.12187740008036295, 'epoch': 3.0}


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/15 [00:00<?, ?it/s]

In [9]:
""" Running the Program (English - French)"""

tune_en_fr_scores = [
    BLANC_tune_translation(
        sentences,
        translations,
        mbert_checkpoint,
        mbert_model,
        mbert_tokenizer,
        device=DEVICE,
    )
    for sentences, translations in tqdm(
        zip(en_fr_sentences, en_fr_translations), total=len(en_fr_sentences)
    )
]

# Saving the results
en_fr_data = {}
en_fr_data["BLANC_tune_en_fa_translation"] = tune_en_fr_scores
add_results_to_json(en_fr_data)

  0%|          | 0/300 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/15 [00:00<?, ?it/s]

{'train_runtime': 6.6442, 'train_samples_per_second': 18.061, 'train_steps_per_second': 2.258, 'train_loss': 0.3151547114054362, 'epoch': 3.0}


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

{'train_runtime': 9.1042, 'train_samples_per_second': 13.181, 'train_steps_per_second': 1.648, 'train_loss': 0.32178007761637367, 'epoch': 3.0}


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/21 [00:00<?, ?it/s]

{'train_runtime': 14.3642, 'train_samples_per_second': 10.443, 'train_steps_per_second': 1.462, 'train_loss': 0.1941494033450172, 'epoch': 3.0}


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000019B376846A0>>
Traceback (most recent call last):
  File "c:\Users\Liora\anaconda3\envs\blanc\lib\site-packages\ipykernel\ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
""" Running the Program (English - Persian)"""

tune_en_fa_scores = [
    BLANC_tune_translation(
        sentences,
        translations,
        mbert_checkpoint,
        mbert_model,
        mbert_tokenizer,
        device=DEVICE,
    )
    for sentences, translations in tqdm(
        zip(en_fa_sentences, en_fa_translations), total=len(en_fa_sentences)
    )
]

# Saving the results
en_fa_data = {}
en_fa_data["BLANC_tune_en_fa_translation"] = tune_en_fa_scores
add_results_to_json(en_fa_data)